<a href="https://colab.research.google.com/github/CalebBrunton2/mgmt467-analytics-portfolio/blob/main/Unit2_Lab2_PromptStudio_Tasks5onwards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🤖 MGMT 467 - Unit 2 Lab 2: Prompt Studio — Feature Engineering & Beyond

**Date:** 2025-10-16  
This notebook continues from Task 5 onward, focusing on feature engineering and model iteration using AI-assisted prompt design.

You'll continue to:
- Generate SQL using prompt templates
- Build and test new features
- Retrain and evaluate your ML model
- Reflect on the effect of engineered features



## Task 5.0: Bucket a Continuous Feature

**🎯 Goal:** Group 'total_minutes' into categories: low, medium, high.  
**📌 Requirements:** Use CASE WHEN or IF statements to create 'watch_time_bucket'.

---

### 🧠 Prompt Template  
> Write SQL that creates a new column watch_time_bucket based on total_minutes thresholds (<100, 100–300, >300).

---

### 👩‍🏫 Example Prompt  
> Create a new column watch_time_bucket with values 'low', 'medium', or 'high' based on total_minutes.

---

### 🔍 Exploration  
How does churn rate vary across these buckets?


Prompt to Gemini:
Write a BigQuery SQL query that adds a new column called watch_time_bucket
to categorize users based on total_minutes.
Use thresholds:
  - 'low' for total_minutes < 100,
  - 'medium' for 100 ≤ total_minutes ≤ 300,
  - 'high' for total_minutes > 300.
Save the result as a new table churn_features_bucketed in my project.


In [ ]:
%%bigquery --project original-wonder-471819-n2
CREATE OR REPLACE TABLE `original-wonder-471819-n2.netflix.churn_features_bucketed` AS
SELECT
  *,
  CASE
    WHEN total_minutes < 100 THEN 'low'
    WHEN total_minutes BETWEEN 100 AND 300 THEN 'medium'
    ELSE 'high'
  END AS watch_time_bucket
FROM `original-wonder-471819-n2.netflix.churn_features_clean`;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT
  watch_time_bucket,
  COUNT(*) AS users,
  AVG(CAST(churn_label AS FLOAT64)) AS churn_rate
FROM `original-wonder-471819-n2.netflix.churn_features_bucketed`
GROUP BY watch_time_bucket
ORDER BY churn_rate DESC;



## Task 5.1: Create a Binary Flag Feature

**🎯 Goal:** Add a binary column flag_binge (1 if total_minutes > 500).  
**📌 Requirements:** Use IF logic to create a binary column in SQL.

---

### 🧠 Prompt Template  
> Write a SQL query that adds flag_binge = 1 if total_minutes > 500, else 0.

---

### 👩‍🏫 Example Prompt  
> Add a binary column flag_binge to identify binge-watchers.

---

### 🔍 Exploration  
Are binge-watchers more or less likely to churn?



## Task 5.2: Create an Interaction Term

**🎯 Goal:** Create plan_region_combo by combining plan_tier and region.  
**📌 Requirements:** Use CONCAT or STRING functions.

---

### 🧠 Prompt Template  
> Generate SQL to create a new column by combining plan_tier and region with an underscore.

---

### 👩‍🏫 Example Prompt  
> Create a column called plan_region_combo as CONCAT(plan_tier, '_', region).

---

### 🔍 Exploration  
Which plan-region combos have highest churn?


Prompt to Gemini:
Write a BigQuery SQL query that creates a binary column named flag_binge
to identify users who have total_minutes > 500.
If total_minutes > 500, flag_binge = 1; otherwise, 0.
Save the results as a new table called churn_features_binge in my project.


In [ ]:
%%bigquery --project original-wonder-471819-n2
CREATE OR REPLACE TABLE `original-wonder-471819-n2.netflix.churn_features_binge` AS
SELECT
  *,
  IF(total_minutes > 500, 1, 0) AS flag_binge
FROM `original-wonder-471819-n2.netflix.churn_features_bucketed`;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT
  flag_binge,
  COUNT(*) AS users,
  AVG(CAST(churn_label AS FLOAT64)) AS churn_rate
FROM `original-wonder-471819-n2.netflix.churn_features_binge`
GROUP BY flag_binge
ORDER BY flag_binge DESC;



## Task 5.3: Add Missingness Indicator Flags

**🎯 Goal:** Add binary flags to capture NULL values in age_band and avg_rating.  
**📌 Requirements:** Use IS NULL logic to create new flag columns.

---

### 🧠 Prompt Template  
> Create a new column is_missing_[col_name] that is 1 when column is NULL, else 0.

---

### 👩‍🏫 Example Prompt  
> Add is_missing_age that flags rows where age_band IS NULL.

---

### 🔍 Exploration  
Do missing values correlate with churn?


Prompt to Gemini:
Write a BigQuery SQL query that adds binary columns is_missing_age and is_missing_rating
to flag missing values. Each column should equal 1 when the corresponding value is NULL, else 0.
Save the new table as churn_features_missing in my project.


In [ ]:
%%bigquery --project original-wonder-471819-n2
CREATE OR REPLACE TABLE `original-wonder-471819-n2.netflix.churn_features_missing` AS
SELECT
  *,
  IF(age_band IS NULL, 1, 0) AS is_missing_age,
  -- keep this as a placeholder, even though avg_rating doesn’t exist yet
  IF(NULL IS NULL, 0, 0) AS is_missing_rating
FROM `original-wonder-471819-n2.netflix.churn_features_binge`;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT
  is_missing_age,
  COUNT(*) AS users,
  AVG(CAST(churn_label AS FLOAT64)) AS churn_rate
FROM `original-wonder-471819-n2.netflix.churn_features_missing`
GROUP BY is_missing_age
ORDER BY is_missing_age DESC;



## Task 5.4: Create Time-Based Features (Optional)

**🎯 Goal:** Add a column days_since_last_login.  
**📌 Requirements:** Use DATE_DIFF with CURRENT_DATE and last_login_date.

---

### 🧠 Prompt Template  
> Write SQL to create a column showing days since last login using DATE_DIFF.

---

### 👩‍🏫 Example Prompt  
> Add a column days_since_last_login = DATE_DIFF(CURRENT_DATE(), last_login_date, DAY).

---

### 🔍 Exploration  
Does login recency affect churn rate?


Prompt to Gemini:
Write a BigQuery SQL query that adds a new column called days_since_last_login
to my churn_features_missing table. Use DATE_DIFF(CURRENT_DATE(), last_login_date, DAY)
to calculate the number of days since the user last logged in.
Save the result as churn_features_time in my project.


In [ ]:
%%bigquery --project original-wonder-471819-n2
CREATE OR REPLACE TABLE `original-wonder-471819-n2.netflix.churn_features_time` AS
SELECT
  *,
  DATE_DIFF(CURRENT_DATE(), last_login_date, DAY) AS days_since_last_login
FROM `original-wonder-471819-n2.netflix.churn_features_missing`;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT
  ROUND(AVG(days_since_last_login), 1) AS avg_days_since_login,
  AVG(CAST(churn_label AS FLOAT64)) AS churn_rate
FROM `original-wonder-471819-n2.netflix.churn_features_time`
GROUP BY
  CASE
    WHEN days_since_last_login < 30 THEN 'Active (<30 days)'
    WHEN days_since_last_login BETWEEN 30 AND 180 THEN 'Inactive (1–6 months)'
    ELSE 'Dormant (>6 months)'
  END
ORDER BY avg_days_since_login;



## Task 5.5: Assemble Enhanced Feature Table

**🎯 Goal:** Create churn_features_enhanced with all engineered columns.  
**📌 Requirements:** Include all prior features + engineered columns.

---

### 🧠 Prompt Template  
> Generate SQL to create churn_features_enhanced with new columns: watch_time_bucket, plan_region_combo, flag_binge, etc.

---

### 👩‍🏫 Example Prompt  
> Build a new table churn_features_enhanced with all original features + engineered ones.

---

### 🔍 Exploration  
Are row counts stable? Any NULLs introduced?


Prompt to Gemini:
Generate a BigQuery SQL statement that creates a new table named
churn_features_enhanced including all original features plus engineered columns:
watch_time_bucket, plan_region_combo, flag_binge, is_missing_age, is_missing_rating,
last_login_date, and days_since_last_login.
Source: original-wonder-471819-n2.netflix.churn_features_time


In [ ]:
%%bigquery --project original-wonder-471819-n2
CREATE OR REPLACE TABLE `original-wonder-471819-n2.netflix.churn_features_enhanced` AS
SELECT
  -- core features
  user_id,
  region,
  plan_tier,
  age_band,
  total_minutes,
  CAST(churn_label AS BOOL) AS churn_label,

  -- engineered features from earlier steps
  watch_time_bucket,                               -- from Task 5.0
  IF(total_minutes > 500, 1, 0) AS flag_binge,     -- from Task 5.1 (recomputed defensively)

  -- composite/interaction feature (Task 5.2)
  CONCAT(
    COALESCE(CAST(plan_tier AS STRING), 'unknown'),
    '_',
    COALESCE(CAST(region AS STRING), 'unknown')
  ) AS plan_region_combo,

  -- missingness indicators (Task 5.3)
  IF(age_band IS NULL, 1, 0) AS is_missing_age,
  -- rating not in schema; keep a placeholder flag for schema stability
  0 AS is_missing_rating,

  -- time features (Task 5.4)
  last_login_date,
  days_since_last_login
FROM `original-wonder-471819-n2.netflix.churn_features_time`;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT
  (SELECT COUNT(*) FROM `original-wonder-471819-n2.netflix.churn_features_time`)  AS src_rows,
  (SELECT COUNT(*) FROM `original-wonder-471819-n2.netflix.churn_features_enhanced`) AS enhanced_rows;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT
  COUNTIF(watch_time_bucket IS NULL) AS null_watch_time_bucket,
  COUNTIF(plan_region_combo IS NULL) AS null_plan_region_combo,
  COUNTIF(flag_binge IS NULL)        AS null_flag_binge,
  COUNTIF(is_missing_age IS NULL)    AS null_is_missing_age,
  COUNTIF(is_missing_rating IS NULL) AS null_is_missing_rating,
  COUNTIF(days_since_last_login IS NULL) AS null_days_since_last_login
FROM `original-wonder-471819-n2.netflix.churn_features_enhanced`;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT *
FROM `original-wonder-471819-n2.netflix.churn_features_enhanced`
LIMIT 10;



## Task 6: Retrain Model on Engineered Features

**🎯 Goal:** Train a logistic regression model using churn_features_enhanced.  
**📌 Requirements:** Use BQML logistic_reg model with new feature columns.

---

### 🧠 Prompt Template  
> Write CREATE MODEL SQL using enhanced features including flags and buckets.

---

### 👩‍🏫 Example Prompt  
> Retrain churn_model_enhanced using watch_time_bucket, flag_binge, plan_region_combo.

---

### 🔍 Exploration  
Does model accuracy improve?


Prompt to Gemini:
Write a BigQuery CREATE MODEL statement for a logistic regression that predicts churn_label
using the enhanced feature table original-wonder-471819-n2.netflix.churn_features_enhanced.
Include categorical columns (strings) directly so BQML encodes them automatically.
Set input_label_cols to churn_label. Name the model churn_model_enhanced.


In [ ]:
%%bigquery --project original-wonder-471819-n2
CREATE OR REPLACE MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`
OPTIONS(
  model_type = 'logistic_reg',
  input_label_cols = ['churn_label']
) AS
SELECT
  -- label
  CAST(churn_label AS BOOL) AS churn_label,

  -- numeric features
  total_minutes,
  flag_binge,
  is_missing_age,
  is_missing_rating,
  days_since_last_login,

  -- categorical features (BQML will one-hot encode strings)
  region,
  plan_tier,
  age_band,
  watch_time_bucket,
  plan_region_combo
FROM `original-wonder-471819-n2.netflix.churn_features_enhanced`;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT *
FROM ML.TRAINING_INFO(MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`);


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT *
FROM ML.EVALUATE(MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`);


In [ ]:
%%bigquery --project original-wonder-471819-n2
WITH base AS (
  SELECT 'baseline' AS model, *
  FROM ML.EVALUATE(MODEL `original-wonder-471819-n2.netflix.churn_model_basic`)
),
enh AS (
  SELECT 'enhanced' AS model, *
  FROM ML.EVALUATE(MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`)
)
SELECT
  model, accuracy, precision, recall, f1_score, roc_auc, log_loss
FROM base
UNION ALL
SELECT
  model, accuracy, precision, recall, f1_score, roc_auc, log_loss
FROM enh
ORDER BY model;



## Task 7: Compare Model Performance

**🎯 Goal:** Compare base model vs enhanced model using ML.EVALUATE.  
**📌 Requirements:** Use same evaluation query for both models.

---

### 🧠 Prompt Template  
> Write a SQL query to evaluate churn_model_enhanced and compare with churn_model.

---

### 👩‍🏫 Example Prompt  
> Compare ML.EVALUATE output from both models side-by-side.

---

### 🔍 Exploration  
Which features made the most difference?


Prompt to Gemini:
Write BigQuery SQL to evaluate two models (churn_model_basic and churn_model_enhanced)
using ML.EVALUATE and show their metrics side-by-side. Then add a second query that
computes metric deltas (enhanced - baseline). Finally, list the top absolute weights
from churn_model_enhanced to see which features mattered most.


In [ ]:
%%bigquery --project original-wonder-471819-n2
WITH w AS (
  SELECT
    feature,
    -- Extract the base feature name before '=something' if present
    COALESCE(REGEXP_EXTRACT(feature, r'^([^=]+)'), feature) AS base_feature,
    weight
  FROM ML.WEIGHTS(MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`)
  WHERE feature NOT IN ('bias')
)
SELECT
  base_feature,
  COUNT(*)                         AS n_terms,
  AVG(ABS(weight))                 AS avg_abs_weight,
  MAX(ABS(weight))                 AS max_abs_weight
FROM w
GROUP BY base_feature
ORDER BY avg_abs_weight DESC
LIMIT 15;


In [ ]:
%%bigquery --project original-wonder-471819-n2
WITH base AS (
  SELECT 'baseline' AS model, *
  FROM ML.EVALUATE(MODEL `original-wonder-471819-n2.netflix.churn_model_basic`)
),
enh AS (
  SELECT 'enhanced' AS model, *
  FROM ML.EVALUATE(MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`)
)
SELECT
  model,
  accuracy, precision, recall, f1_score, roc_auc, log_loss
FROM base
UNION ALL
SELECT
  model,
  accuracy, precision, recall, f1_score, roc_auc, log_loss
FROM enh
ORDER BY model;


In [ ]:
%%bigquery --project original-wonder-471819-n2
WITH base AS (
  SELECT * FROM ML.EVALUATE(MODEL `original-wonder-471819-n2.netflix.churn_model_basic`)
),
enh AS (
  SELECT * FROM ML.EVALUATE(MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`)
)
SELECT
  (enh.accuracy - base.accuracy)   AS d_accuracy,
  (enh.precision - base.precision) AS d_precision,
  (enh.recall   - base.recall)     AS d_recall,
  (enh.f1_score - base.f1_score)   AS d_f1,
  (enh.roc_auc  - base.roc_auc)    AS d_roc_auc,
  (enh.log_loss - base.log_loss)   AS d_log_loss
FROM base, enh;


In [ ]:
%%bigquery --project original-wonder-471819-n2
SELECT
  feature,
  category,
  weight
FROM ML.WEIGHTS(MODEL `original-wonder-471819-n2.netflix.churn_model_enhanced`)
WHERE feature NOT IN ('bias')
ORDER BY ABS(weight) DESC
LIMIT 25;



🎯 Feature impact:
Top absolute weights suggest the most influential terms (e.g., watch_time_bucket, plan_region_combo,
days_since_last_login, flag_binge). Categorical expansions (region, plan_tier, age_band) appear as
one-hot terms like region=West or plan_tier=Premium.

📌 Takeaway:
If recall improved with acceptable precision, the engineered features helped catch more true churners.
If precision improved, targeting can be tighter with fewer false alarms. Tune threshold next based on
the business cost of false positives vs. false negatives.
